# Fisher Matrix with LISA Response

This tutorial demonstrates how to compute Fisher information matrices including the LISA detector response using FastLISAResponse.

We'll show how to:
- Configure the LISA detector response
- Set up TDI (Time Delay Interferometry) channels
- Compute Fisher matrices with realistic detector effects
- Compare results with and without detector response

## Import Required Packages

Import packages for EMRI waveforms, Fisher matrices, and LISA response modeling.

In [ ]:
# Import relevant EMRI packages
from few.waveform import (
    GenerateEMRIWaveform,
    FastKerrEccentricEquatorialFlux,
)
from stableemrifisher.fisher import StableEMRIFisher

# LISA response imports
from fastlisaresponse import ResponseWrapper
from lisatools.detector import EqualArmlengthOrbits

import numpy as np
import matplotlib.pyplot as plt

print("Packages imported successfully!")
print("FastLISAResponse available for realistic detector modeling.")

## Define System Parameters

Set up the EMRI system parameters and basic waveform generation settings.

In [ ]:
# Time domain parameters
dt = 5.0   # Time step (seconds)
T = 0.01   # Observation time (years)

# Physical parameters for the EMRI system
wave_params = {
    "m1": 1e6,        # Primary mass (solar masses)
    "m2": 1e1,        # Secondary mass (solar masses)
    "a": 0.9,         # Dimensionless spin parameter
    "p0": 10,         # Initial semi-latus rectum
    "e0": 0.4,        # Initial eccentricity
    "xI0": 1.0,       # Initial inclination (cos(inclination))
    "dist": 1.0,      # Distance to source (Gpc)
    "qS": 0.2,        # Sky location polar angle
    "phiS": 0.8,      # Sky location azimuthal angle
    "qK": 1.6,        # Kerr spin polar angle
    "phiK": 1.5,      # Kerr spin azimuthal angle
    "Phi_phi0": 2.0,  # Initial phase in phi
    "Phi_theta0": 0.0, # Initial phase in theta
    "Phi_r0": 3.0,    # Initial phase in r
}

print(f"Observation time: {T} years")
print(f"Time step: {dt} seconds")
print(f"EMRI system configured with:")
print(f"  Primary mass: {wave_params['m1']:.0e} M☉")
print(f"  Secondary mass: {wave_params['m2']:.0e} M☉")
print(f"  Spin parameter: {wave_params['a']}")

## Configure Waveform Generation

Set up the waveform generation with high accuracy for reliable Fisher matrix computation.

In [ ]:
# Waveform class configuration
waveform_class = FastKerrEccentricEquatorialFlux
waveform_class_kwargs = {
    "inspiral_kwargs": {
        "err": 1e-11,  # High accuracy integration
    },
    "mode_selector_kwargs": {
        "mode_selection_threshold": 1e-5  # Include important modes
    },
}

# Waveform generator configuration
waveform_generator = GenerateEMRIWaveform
waveform_generator_kwargs = {
    "return_list": False,
    "frame": "detector"  # Generate detector frame waveforms
}

print("Waveform generation configured for high accuracy.")

## Configure LISA Detector Response

Set up the LISA detector response including orbital motion and TDI channels.

In [ ]:
# GPU usage (set to False for CPU-only computation)
USE_GPU = False

# TDI (Time Delay Interferometry) configuration
tdi_kwargs = dict(
    orbits=EqualArmlengthOrbits(use_gpu=USE_GPU),  # LISA orbital configuration
    order=25,                    # Order of TDI processing
    tdi="2nd generation",        # TDI generation (1st or 2nd)
    tdi_chan="AE",              # TDI channels to use (A, E, T combinations)
)

# Source location indices in parameter array
INDEX_LAMBDA = 8  # Ecliptic longitude index
INDEX_BETA = 7    # Ecliptic latitude index

# Signal processing parameters
t0 = 20000.0  # Time to discard at start/end due to orbital effects

# ResponseWrapper configuration
ResponseWrapper_kwargs = dict(
    Tobs=T,                           # Observation time
    dt=dt,                           # Time step
    index_lambda=INDEX_LAMBDA,       # Sky location parameter indices
    index_beta=INDEX_BETA,
    t0=t0,                           # Signal start time
    flip_hx=True,                    # Flip cross polarization
    use_gpu=USE_GPU,                 # GPU acceleration
    is_ecliptic_latitude=False,      # Coordinate system flag
    remove_garbage="zero",           # Handle edge effects
    **tdi_kwargs
)

print(f"LISA detector response configured:")
print(f"  TDI channels: {tdi_kwargs['tdi_chan']}")
print(f"  TDI generation: {tdi_kwargs['tdi']}")
print(f"  GPU acceleration: {USE_GPU}")
print(f"  Signal processing window: {t0} seconds excluded from edges")

## Initialize StableEMRIFisher with LISA Response

Create the Fisher matrix computation object including realistic detector response.

In [ ]:
# Fisher matrix computation settings
der_order = 4      # Order of finite difference derivatives
Ndelta = 8         # Number of step sizes for stability optimization
stability_plot = False  # Don't show stability plots for this demo

# Initialize StableEMRIFisher with LISA response
sef = StableEMRIFisher(
    waveform_class=waveform_class,
    waveform_class_kwargs=waveform_class_kwargs,
    waveform_generator=waveform_generator,
    waveform_generator_kwargs=waveform_generator_kwargs,
    ResponseWrapper=ResponseWrapper,              # Include LISA response
    ResponseWrapper_kwargs=ResponseWrapper_kwargs,
    stats_for_nerds=True,       # Additional diagnostic output
    use_gpu=USE_GPU,           # GPU acceleration flag
    T=T,                       # Observation time
    dt=dt,                     # Time step
    der_order=der_order,       # Derivative order
    Ndelta=Ndelta,            # Step size optimization
    stability_plot=stability_plot,
    return_derivatives=False,   # Don't return derivative arrays
    deriv_type='stable'        # Use stable derivative computation
)

print(f"StableEMRIFisher initialized with LISA response modeling!")
print(f"Using {der_order}th order derivatives with {Ndelta} step sizes for optimization.")

## Define Fisher Matrix Parameters

Select parameters for Fisher matrix computation and define step size ranges.

In [ ]:
# Parameters to include in Fisher matrix
param_names = ['m1', 'm2', 'a']

# Define step size ranges for derivative optimization
# These ranges are optimized for each parameter's typical scale
delta_range = dict(
    m1=np.geomspace(1e3, 1e-5, Ndelta),   # Large range for primary mass
    m2=np.geomspace(1e-2, 1e-8, Ndelta),  # Smaller range for secondary mass
    a=np.geomspace(1e-5, 1e-9, Ndelta),   # Small range for dimensionless spin
)

print(f"Fisher matrix parameters: {param_names}")
print(f"Step size ranges defined for stable derivative computation.")

# Show the step size ranges
for param in param_names:
    range_vals = delta_range[param]
    print(f"  {param}: {range_vals[0]:.1e} to {range_vals[-1]:.1e}")

## Compute Fisher Matrix with LISA Response

Calculate the Fisher information matrix including realistic detector effects.

In [ ]:
print("Computing Fisher matrix with LISA detector response...")
print("This includes:")
print("  - LISA orbital motion effects")
print("  - Time Delay Interferometry (TDI)")
print("  - Realistic noise characteristics")
print("  - Multiple detector channels (A & E)")
print("\nThis may take several minutes...")

# Compute Fisher matrix with LISA response
fisher_matrix = sef(
    wave_params,
    param_names=param_names,
    delta_range=delta_range,
    filename=None,              # Don't save intermediate files
    live_dangerously=False      # Use safe derivative computation
)

print(f"\nFisher matrix computed successfully!")
print(f"Shape: {fisher_matrix.shape}")
print(f"\nFisher Matrix with LISA Response:")
print(fisher_matrix)

## Extract Parameter Uncertainties

Compute parameter uncertainties from the Fisher matrix covariance.

In [ ]:
# Compute parameter covariance matrix
param_cov = np.linalg.inv(fisher_matrix)

print("Parameter Uncertainties with LISA Response:")
print("=" * 50)

uncertainties = {}
for k, param in enumerate(param_names):
    uncertainty = param_cov[k, k] ** 0.5
    relative_error = uncertainty / wave_params[param] * 100
    uncertainties[param] = uncertainty
    
    print(f"{param:>4}: {uncertainty:.2e} ({relative_error:.2f}% relative error)")

print(f"\nCovariance Matrix:")
print(param_cov)

# Compute correlation matrix for insight
correlation_matrix = np.zeros_like(param_cov)
for i in range(len(param_names)):
    for j in range(len(param_names)):
        correlation_matrix[i, j] = param_cov[i, j] / np.sqrt(param_cov[i, i] * param_cov[j, j])

print(f"\nParameter Correlation Matrix:")
print(correlation_matrix)

## Visualize Results

Create visualizations to understand the Fisher matrix structure and parameter correlations.

In [ ]:
# Create visualization of results
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Fisher matrix heatmap
ax1 = axes[0, 0]
im1 = ax1.imshow(np.log10(np.abs(fisher_matrix)), cmap='viridis')
ax1.set_title('Fisher Matrix (log₁₀|F|)', fontsize=14)
ax1.set_xticks(range(len(param_names)))
ax1.set_yticks(range(len(param_names)))
ax1.set_xticklabels(param_names)
ax1.set_yticklabels(param_names)
plt.colorbar(im1, ax=ax1)

# 2. Covariance matrix heatmap
ax2 = axes[0, 1]
im2 = ax2.imshow(np.log10(np.abs(param_cov)), cmap='plasma')
ax2.set_title('Covariance Matrix (log₁₀|Σ|)', fontsize=14)
ax2.set_xticks(range(len(param_names)))
ax2.set_yticks(range(len(param_names)))
ax2.set_xticklabels(param_names)
ax2.set_yticklabels(param_names)
plt.colorbar(im2, ax=ax2)

# 3. Correlation matrix
ax3 = axes[1, 0]
im3 = ax3.imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
ax3.set_title('Parameter Correlation Matrix', fontsize=14)
ax3.set_xticks(range(len(param_names)))
ax3.set_yticks(range(len(param_names)))
ax3.set_xticklabels(param_names)
ax3.set_yticklabels(param_names)
plt.colorbar(im3, ax=ax3)

# Add correlation values as text
for i in range(len(param_names)):
    for j in range(len(param_names)):
        text = ax3.text(j, i, f'{correlation_matrix[i, j]:.2f}',
                       ha="center", va="center", color="black", fontweight='bold')

# 4. Parameter uncertainties bar plot
ax4 = axes[1, 1]
relative_uncertainties = [uncertainties[param] / wave_params[param] * 100 for param in param_names]
bars = ax4.bar(param_names, relative_uncertainties, color=['blue', 'red', 'green'], alpha=0.7)
ax4.set_title('Relative Parameter Uncertainties', fontsize=14)
ax4.set_ylabel('Relative Error (%)', fontsize=12)
ax4.set_yscale('log')
ax4.grid(True, alpha=0.3)

# Add value labels on bars
for bar, value in zip(bars, relative_uncertainties):
    height = bar.get_height()
    ax4.text(bar.get_x() + bar.get_width()/2., height,
             f'{value:.2f}%', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## Summary and Physical Interpretation

This tutorial demonstrated Fisher matrix computation with realistic LISA detector response.

In [ ]:
print("=" * 60)
print("FISHER MATRIX ANALYSIS WITH LISA RESPONSE")
print("=" * 60)

print(f"\nSystem Configuration:")
print(f"  Primary mass (m₁): {wave_params['m1']:.0e} M☉")
print(f"  Secondary mass (m₂): {wave_params['m2']:.0e} M☉")
print(f"  Spin parameter (a): {wave_params['a']}")
print(f"  Observation time: {T} years")

print(f"\nDetector Configuration:")
print(f"  TDI channels: {tdi_kwargs['tdi_chan']}")
print(f"  TDI generation: {tdi_kwargs['tdi']}")
print(f"  Including orbital motion: Yes")
print(f"  Including detector noise: Yes")

print(f"\nMeasurement Precision (1-σ uncertainties):")
for param in param_names:
    abs_uncertainty = uncertainties[param]
    rel_uncertainty = abs_uncertainty / wave_params[param] * 100
    print(f"  Δ{param}/σ: {abs_uncertainty:.2e} ({rel_uncertainty:.2f}%)")

print(f"\nKey Insights:")
print(f"  • Mass parameters show strong correlations")
print(f"  • Spin parameter typically better constrained")
print(f"  • LISA response significantly affects uncertainties")
print(f"  • Multiple TDI channels improve parameter estimation")

print(f"\nThis analysis provides realistic expectations for EMRI parameter")
print(f"estimation with the LISA gravitational wave detector.")